In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
import kagglehub
dhruvtangri1998_asvspoof_dataset_2019_path = kagglehub.dataset_download('dhruvtangri1998/asvspoof-dataset-2019')

print('Data source import complete.')

100%|██████████| 7.12G/7.12G [04:01<00:00, 31.6MB/s]

Extracting files...


Data source import complete.


In [ ]:
dhruvtangri1998_asvspoof_dataset_2019_path

'/root/.cache/kagglehub/datasets/dhruvtangri1998/asvspoof-dataset-2019/versions/1'

In [1]:
import logging

# Configure logging to write to a file
logging.basicConfig(
    filename='app.log',
    filemode='a',  # 'w' to overwrite, 'a' to append
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)
logger.info("Logging initialized")

In [ ]:
!python train_search.py --layers=8 --init_channels=64 --sinc_scale=mel

In [2]:
logger.debug("Processing model...")

In [3]:
!python train_model.py --arch="Genotype(normal=[('dil_conv_5', 1), ('dil_conv_3', 0), ('dil_conv_5', 1), ('dil_conv_5', 2), ('std_conv_5', 2), ('skip_connect', 3), ('std_conv_5', 2), ('skip_connect', 4)], normal_concat=range(2, 6), reduce=[('max_pool_3', 0), ('std_conv_3', 1), ('dil_conv_3', 0), ('dil_conv_3', 2), ('skip_connect', 0), ('dil_conv_5', 2), ('dil_conv_3', 0), ('avg_pool_3', 1)], reduce_concat=range(2, 6))" --layers=8 --init_channels=64 --sinc_scale=mel

Experiment dir : train-EXP-20250317-114811
03/17 11:48:11 AM args = Namespace(data='/root/.cache/kagglehub/datasets/dhruvtangri1998/asvspoof-dataset-2019/versions/1/LA', valid_freq=1, report_freq=1000, layers=8, init_channels=64, gru_hsize=1024, gru_layers=3, sinc_scale='mel', sinc_kernel=128, is_trainable=False, pre_trained=None, arch="Genotype(normal=[('dil_conv_5', 1), ('dil_conv_3', 0), ('dil_conv_5', 1), ('dil_conv_5', 2), ('std_conv_5', 2), ('skip_connect', 3), ('std_conv_5', 2), ('skip_connect', 4)], normal_concat=range(2, 6), reduce=[('max_pool_3', 0), ('std_conv_3', 1), ('dil_conv_3', 0), ('dil_conv_3', 2), ('skip_connect', 0), ('dil_conv_5', 2), ('dil_conv_3', 0), ('avg_pool_3', 1)], reduce_concat=range(2, 6))", batch_size=32, num_epochs=100, is_mask=True, sr=16000, lr=5e-05, lr_min=2e-05, weight_decay=0.0003, seed=None, comment='train-EXP-20250317-114811', momentum=0.9, unrolled=False, drop_path_prob=0.0, is_rand=False)
****** Initialising Sinc filters as front end ******
No

In [11]:
!python evaluate.py --arch="Genotype(normal=[('dil_conv_5', 1), ('dil_conv_3', 0), ('dil_conv_5', 1), ('dil_conv_5', 2), ('std_conv_5', 2), ('skip_connect', 3), ('std_conv_5', 2), ('skip_connect', 4)], normal_concat=range(2, 6), reduce=[('max_pool_3', 0), ('std_conv_3', 1), ('dil_conv_3', 0), ('dil_conv_3', 2), ('skip_connect', 0), ('dil_conv_5', 2), ('dil_conv_3', 0), ('avg_pool_3', 1)], reduce_concat=range(2, 6))" --model=/content/train-EXP-20250317-114811/models/epoch_49.pth --layers=8 --init_channels=64 --sinc_scale=mel --comment=""

/content/evaluate.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
****** Initialising Sinc filters as front end ******
Normal Cell
6

In [12]:
import numpy as np
from sklearn.metrics import roc_curve

def compute_eer(scores_file):
    # Parse the score file
    bonafide_scores = []
    spoof_scores = []

    with open(scores_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 4:
                continue

            filename, attack_id, label, score = parts
            score = float(score)

            if label == 'bonafide':
                bonafide_scores.append(score)
            else:  # 'spoof'
                spoof_scores.append(score)

    # Create labels and scores arrays
    y_true = [1] * len(bonafide_scores) + [0] * len(spoof_scores)
    y_score = bonafide_scores + spoof_scores

    # Calculate FRR and FAR for different thresholds
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    fnr = 1 - tpr  # False Negative Rate = 1 - True Positive Rate

    # Find the threshold where FAR (FPR) = FRR (FNR)
    eer_threshold = thresholds[np.nanargmin(np.absolute(fnr - fpr))]
    eer = fpr[np.nanargmin(np.absolute(fnr - fpr))]

    return eer, eer_threshold

In [13]:
compute_eer("/content/score-.txt")

(0.028897028897028897, 0.9676753282546997)